<a href="https://colab.research.google.com/github/AnshMittal1811/Pytorch/blob/master/04_Training_a_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Neural networks comprise of layers/modules that perform operations on data. The torch.nn namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In the following sections, we’ll build a neural network to classify images in the FashionMNIST dataset.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Device to train the neural network

We want to be able to train our model on a hardware accelerator like the GPU, if it is available. Let’s check to see if torch.cuda is available, else we continue to use the CPU.

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


## Define the Class

We define our neural network by subclassing ```nn.Module```, and initialize the neural network layers in ```__init__```. Every ```nn.Module``` subclass implements the operations on input data in the ```forward``` method.



In [5]:
class NeuralNetwork(nn.Module):
  def __init__(self): 
    super(NeuralNetwork, self).__init__()

    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(), 
        nn.Linear(512, 512),
        nn.ReLU(), 
        nn.Linear(512, 10),
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

We create an instance of ```NeuralNetwork```, and move it to the ```device```, and print its structure.

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s ```forward```, along with some background operations. Do not call ```model.forward()``` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class. We get the prediction probabilities by passing it through an instance of the nn.Softmax module.



In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)
print(f"Predicted Class: {y_pred}")

Predicted Class: tensor([8], device='cuda:0')


-----------

## Model Layers

Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.



In [9]:
input_img =  torch.rand(3, 28, 28)
print(input_img.size())

torch.Size([3, 28, 28])


## nn.Flatten
We initialize the nn.Flatten layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).



In [10]:
flatten = nn.Flatten()
flat_img = flatten(input_img)
print(flat_img.size())

torch.Size([3, 784])


## nn.Linear 

The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.



In [13]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_img)
print(hidden1.size())

torch.Size([3, 20])


# nn.ReLU

Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use ```nn.ReLU``` between our linear layers, but there’s other activations to introduce non-linearity in your model.



In [14]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2615,  0.1901, -0.0063,  0.2940,  0.0804,  0.0881,  0.0576, -0.2771,
         -0.1235,  0.7038,  0.4804, -0.0816,  0.4987, -0.1458,  0.0797, -0.9553,
         -0.0090, -0.6810,  0.1233, -0.6272],
        [-0.0079,  0.2747, -0.2281,  0.3088, -0.0808,  0.0985,  0.2077, -0.2825,
         -0.1751,  0.6931,  0.2751, -0.3432,  0.3762, -0.0102, -0.1423, -0.8367,
          0.0872, -0.3589,  0.4555, -0.5316],
        [-0.3485, -0.0337, -0.0140,  0.3045,  0.2762,  0.0876,  0.0954,  0.0203,
         -0.3411,  0.2113,  0.2906, -0.0050,  0.4157,  0.2901,  0.3068, -0.7661,
         -0.0366, -0.3191, -0.0357, -0.4231]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.1901, 0.0000, 0.2940, 0.0804, 0.0881, 0.0576, 0.0000, 0.0000,
         0.7038, 0.4804, 0.0000, 0.4987, 0.0000, 0.0797, 0.0000, 0.0000, 0.0000,
         0.1233, 0.0000],
        [0.0000, 0.2747, 0.0000, 0.3088, 0.0000, 0.0985, 0.2077, 0.0000, 0.0000,
         0.6931, 0.2751, 0.0000, 0.3762, 0.0000, 0.00

## nn.Sequential
nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like ```seq_modules```.



In [15]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## nn.Softmax
The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the ```nn.Softmax``` module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. ```dim``` parameter indicates the dimension along which the values must sum to 1.



In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## Model Parameters
Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. Subclassing ```nn.Module``` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s ```parameters()``` or ```named_parameters()``` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.



In [17]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters(): 
  print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0327, -0.0076,  0.0314,  ...,  0.0144,  0.0097, -0.0192],
        [-0.0175, -0.0199,  0.0109,  ..., -0.0098,  0.0008, -0.0356]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0208, -0.0222], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0344, -0.0240,  0.0012,  ...,  0.0387,  0.0272,  0.0322],
        [-0.0145,  0.0208, -0.0432,  ..., -0.0439,  0.0062, -0.0423]],
       device='cuda:0', grad_fn=<Sli

------